In [1]:
from loader import load_classification_dataset
from simple_train import train_evaluate
from pretrain import pretrain_finetune_evaluate

import tensorflow.keras as keras
import numpy as np

In [2]:
ds_path = os.path.join(os.getcwd(), 'datasets')

In [3]:
LSTM_UNITS = 64

def print_results(r):
    train_result, eval_result = r
    print(f'Train | Loss: {train_result[0]:.2f} - Acc: {train_result[1]:.2f}')
    print(f'Eval  | Loss: {eval_result[0]:.2f} - Acc: {eval_result[1]:.2f}')

def print_accs(r):
    r = np.array(r)
    m = np.mean(r)
    std = np.std(r)

    print(f'Accuracy | Mean: {m:.2f} - Std: {std:.2f}')

In [4]:
def lstm_trunk(x):
    return keras.layers.LSTM(LSTM_UNITS)(x)

In [5]:
def lstm_autoencoder(input_x):
    x = keras.layers.LSTM(LSTM_UNITS, name='trunk')(input_x)

    x = keras.layers.RepeatVector(input_x.shape[1])(x)

    x = keras.layers.LSTM(LSTM_UNITS, return_sequences=True)(x)

    return keras.layers.TimeDistributed(keras.layers.Dense(input_x.shape[-1]))(x)

In [6]:
#for ds_name in os.listdir(ds_path):
for ds_name in ['BasicMotions']:
    ds_load_result = load_classification_dataset(ds_path, ds_name)
    print(f'--- Dataset: {ds_name}')
    print()
    print(f'Simple Train >')
    r = train_evaluate(ds_load_result=ds_load_result, trunk=lstm_trunk)
    print_accs(r)

    print()
    print(f'Pretrain and Fine Tune>')

    r = pretrain_finetune_evaluate(ds_load_result=ds_load_result,pretrain_model=lstm_autoencoder)
    print_acc(r)
    print()
    print()

0it [00:00, ?it/s]--- Dataset: BasicMotions

Simple Train >
5it [02:15, 27.11s/it]
Accuracy | Mean: 0.65 - Std: 0.14

Pretrain and Fine Tune>
Train | Loss: 1.43 - Acc: 0.12
Eval  | Loss: 1.28 - Acc: 0.47


